# Weather forecaster 🌦️⛅🌞

Forecast the weather for locations specified by our customer.
Traveler's need good weather forecasts to plan appropriate attire and plan activities.
AI will take on the Meteorologist persona and generate a friendly weather forecast summary using near real-time weather data from [tomorrow.io](https://tomorrow.io).

Instructions for getting started with this Ultralight Backpackers AI trek are on the [GitHub page's README](../README.md). 

# 🧭🥾 Ultralight gear list for AI-driven Weather Forecasts

- 1 JSON fromatted daily weather forcast.  We need the highest quality gear available so we'll use [tomorrow.io](https://www.tomorrow.io/).
- 1 [Semantic Plugin GetLatLon](./plugins-sk/Weather/GetLatLon/skprompt.txt) to help the language model return latitude and longitude in a very specific format
- 1 [Semantic Plugin Summarize](./plugins-sk/Weather/Summarize/skprompt.txt) giving the language model a weather forecaster persona.



> [!IMPORTANT]
> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

# Gather the core tools 🧰

When running the following cell, if asked to "select your kernel" (note this will be referring to the Jupyter notebook's kernel and not Semantic Kernel) then choose `.NET Interactive` from the available menu options.

In [ ]:
// Load some helper functions, e.g. to load values from settings.json
#!import ../config/Settings.cs 
#!import ../plugins/WeatherPlugin.cs
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta1"


# Fire up a kernel 🔥

In [ ]:
using Microsoft.SemanticKernel;
using System;
using System.Threading.Tasks;
using Microsoft.SemanticKernel.Planners;

var builder = new KernelBuilder();
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, tomorrowioApiKey) = Settings.LoadFromFile();
Console.WriteLine($"Using 🧱 Model: {model}");

if (useAzureOpenAI)
    builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAIChatCompletionService(model, apiKey, orgId);

IKernel kernel = builder.Build();
// Load the Plugins
kernel.ImportFunctions(new WeatherSkill(tomorrowioApiKey), "WeatherPlugin");
kernel.ImportSemanticFunctionsFromDirectory("plugins-sk", "Weather");

var plannerConfig = new SequentialPlannerConfig();
//plannerConfig.ExcludedFunctions.Add("Summarize");
var planner = new SequentialPlanner(kernel, plannerConfig);


# Use 🗺️ planner and several 🔌 PLUGINs with 📄 input of your choice

In [ ]:
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Plugins.Core;

TimeSpan startTime = TimeSpan.FromTicks(DateTime.Now.Ticks);
string[] sentences = new string[] {
    "What is the weather forecast for Austin, Texas?",
    "What is the weather forecast for Austin, Texas tomorrow?",
    "What is the weather forecast for Austin, Texas on Friday?",
    "What is the weather forecast for College Station, TX?",
    "What will the weather be like in Chicago tomorrow?",
    "Will it rain this week in Seattle, Washington?"
};

// create a random int between 0 and 4
var random = new Random();
var randomIndex = random.Next(0, 5);
var plan = await planner.CreatePlanAsync(sentences[randomIndex]);
Console.WriteLine($"Using 🗺️ Plan: {plan.Description}");
Console.WriteLine($"{plan.Display()}");


# ➡️Section Hike the 🗺️plan, using the 🧰tools we carefully packed

In [ ]:
using Microsoft.AspNetCore.Html;
using Markdig;

var myResult = await plan.InvokeAsync(kernel);
var myPipeline = new MarkdownPipelineBuilder().UseAdvancedExtensions().Build();
var myResult2HTML = Markdown.ToHtml("# The forecast?\n" + myResult.GetValue<string>(), myPipeline);
var myHTMLContent = new HtmlContentBuilder();

myHTMLContent.AppendHtml(myResult2HTML);
myHTMLContent